<h1> Mobilenet Model Training </h1>

<h3> In this notebook, we will train a Mobilenet binary classifier that can classify images into "contains fish" and "does not contain fish". </h3>

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/root/alok/repos/cv_research/alok')

import glob
import json
import os
import numpy as np
from PIL import Image
from random import shuffle

import keras
from keras.models import Model
from keras.applications.mobilenet import MobileNet
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras import backend as K
from keras.backend import tf as ktf

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.io import imread
from skimage.transform import resize
import time

%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"]="1"
np.random.seed(0)

<h1> First step: Prepare the data </h1>

In [ ]:
# get data ready

# define input locations
nonzero_crop_image_dir = '/root/data/alok/filtration_classifier_data/nonzero_crops/images_resized'
nonzero_crop_image_fs = glob.glob(os.path.join(nonzero_crop_image_dir, '*.jpg'))
nonzero_crop_metadata = [1] * len(nonzero_crop_image_fs)

zero_crop_image_dir = '/root/data/alok/filtration_classifier_data/zero_crops/images_resized'
zero_crop_image_fs = glob.glob(os.path.join(zero_crop_image_dir, '*.jpg'))
zero_crop_metadata = [0] * len(nonzero_crop_image_fs)


# create the full dataset
nonzero_crops_dataset = list(zip(nonzero_crop_image_fs, nonzero_crop_metadata))
zero_crops_dataset = list(zip(zero_crop_image_fs, zero_crop_metadata))

train_pct, val_pct, test_pct = 0.6, 0.1, 0.3

nonzero_N, zero_N = len(nonzero_crops_dataset), len(zero_crops_dataset)


training_dataset = nonzero_crops_dataset[:int(train_pct*nonzero_N)] + zero_crops_dataset[:int(train_pct*zero_N)]
validation_dataset = nonzero_crops_dataset[int(train_pct*nonzero_N):int((train_pct+val_pct)*nonzero_N)] + zero_crops_dataset[int(train_pct*zero_N):int((train_pct+val_pct)*zero_N)]
testing_dataset = nonzero_crops_dataset[int((train_pct+val_pct)*nonzero_N):] + zero_crops_dataset[int((train_pct+val_pct)*zero_N):]

shuffle(training_dataset)
shuffle(validation_dataset)
shuffle(testing_dataset)





In [ ]:
N_train, N_val, N_test = len(training_dataset), len(validation_dataset), len(testing_dataset)

X_train = np.empty((N_train, 224, 224, 3))
y_train = np.empty(N_train)
X_val = np.empty((N_val, 224, 224, 3))
y_val = np.empty(N_val)
X_test = np.empty((N_test, 224, 224, 3))
y_test = np.empty(N_test)

for i, data_point in enumerate(training_dataset):
    image_f, cls = data_point
    im = Image.open(image_f)
    X_train[i, :] = np.array(im) / 255.0
    y_train[i] = cls
    
print('Training matrix populated')
    
for i, data_point in enumerate(validation_dataset):
    image_f, cls = data_point
    im = Image.open(image_f)
    X_val[i, :] = np.array(im) / 255.0
    y_val[i] = cls
    
print('Validation matrix populated')
    
for i, data_point in enumerate(testing_dataset):
    image_f, cls = data_point
    im = Image.open(image_f)
    X_test[i, :] = np.array(im) / 255.0
    y_test[i] = cls
    
    
print('Testing matrix populated')

<h1> Define The Model </h1>

In [ ]:
# Generate a model with all layers (with top)
mobilenet = MobileNet(input_shape=(224, 224, 3))
x = Dense(1, activation='sigmoid')(mobilenet.layers[-1].output)
model = Model(input=mobilenet.input, output=x)
model.summary()

In [ ]:
# optimizer = keras.optimizers.adam(lr=0.0001, decay=0.1)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
results = model.evaluate(X_test, y_test)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
threshold = 0.5
false_positive_cnt = 0
true_positive_cnt = 0
false_negative_cnt = 0
for p, gt in zip(predictions, y_test):
    if gt == 1 and p < threshold:
        false_positive_cnt += 1
    if gt == 0 and p < threshold:
        true_positive_cnt += 1
    if gt == 0 and p > threshold:
        false_negative_cnt += 1
        

In [ ]:
false_positive_cnt / len([k for k in y_test if k == 1])

In [ ]:
true_positive_cnt / len([k for k in y_test if k == 0])

In [ ]:
false_negative_cnt / len([p for p in predictions if p > threshold])

In [ ]:
model.save_weights('model.h5')

<h1> Define Generator </h1>

In [ ]:
def data_generator(dset, steps_per_epoch, batch_size, input_shape):
    N = len(dset)
    i = 0
    while True:
        x_batch = np.empty((batch_size, *input_shape))
        y_batch = np.empty(batch_size)
        batch = dset[batch_size * i : min(batch_size * (i + 1), N)]
        for idx, d in enumerate(batch):
            image_f, metadata = d
            im = Image.open(image_f)
            
            # normalize image
            
            x_batch[idx, :] = im
            y_batch[idx] = metadata
        
        i += 1
        if i > steps_per_epoch:
            i = 0
        yield x_batch, y_batch


<h1> Train Model </h1>

In [ ]:
# optimizer = keras.optimizers.adam(lr=0.0001, decay=0.1)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 25
steps_per_epoch = int(len(training_dataset)/BATCH_SIZE) + 1
gen = data_generator(training_dataset, steps_per_epoch, BATCH_SIZE, (224, 224, 3))

In [ ]:
model.fit_generator(gen, steps_per_epoch=steps_per_epoch, epochs=100, verbose=True)

In [ ]:
t = time.time()
batches = len(testing_dataset) // BATCH_SIZE
eval_gen = data_generator(testing_dataset, np.inf, BATCH_SIZE, (224, 224, 3))
scores = model.evaluate_generator(eval_gen, batches)
print(scores)
print(time.time() - t)

In [ ]:
eval_gen = data_generator(testing_dataset, np.inf, BATCH_SIZE, (224, 224, 3))
predictions = model.predict_generator(eval_gen, batches)

In [ ]:
ground_truth_classes = [i[1] for i in testing_dataset ]

In [ ]:
threshold = 0.5
false_positive_cnt = 0
filter_cnt = 0
for p, gt in zip(predictions, ground_truth_classes):
    if gt == 1 and p[0] < threshold:
        false_positive_cnt += 1
    if gt == 0 and p[0] < threshold:
        filter_cnt += 1
        

In [ ]:
print(false_positive_cnt / len([g for g in ground_truth_classes if g == 1]))
print(filter_cnt / len([g for g in ground_truth_classes if g == 0]))

In [ ]:
false_positive_cnt

In [ ]:
true_negative_cnt

In [ ]:
ground_truth_classes = []
for i in range(1025):
    c = json.load(open(testing_dataset[i][1]))['model']
    ground_truth_classes.append(1 if c == 'contains_fish' else 0)

<h1> Quick test </h1>

In [ ]:
files = glob.glob('/root/data/alok/filtration_classifier_data/fish_present/images_resized/*.jpg')

In [ ]:
adhoc_dataset = []
for i, f in enumerate(files):
    adhoc_dataset.append((f, dataset[i][1]))
    

In [ ]:
BATCH_SIZE = 1
adhoc_gen = data_generator(adhoc_dataset, np.inf, BATCH_SIZE, (224, 224, 3))
predictions = model.predict_generator(adhoc_gen, len(adhoc_dataset))

In [ ]:
pass_cnt = 0
for p in predictions:
    if p > 0.5:
        pass_cnt += 1

        

In [ ]:
pass_cnt

In [ ]:
len(predictions)

In [ ]:
762 / 25

In [ ]:
(750 - 476) / 750.

In [ ]:
len(files) / 25

In [ ]:
(1500 - 937) / 1500